In [7]:
file_path = 'API_KEY.txt'

with open(file_path, 'r') as file:
        api_key = file.read()

In [8]:
import base64
import requests

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "foto/hema.jpg"

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

payload = {
  "model": "gpt-4o-mini",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Geef alle producten die gekocht zijn en de prijs. Output format is json."
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

{'id': 'chatcmpl-AheRLfw9x4sf9tI1DuUXhw5QfBUL9', 'object': 'chat.completion', 'created': 1734966919, 'model': 'gpt-4o-mini-2024-07-18', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Hier is de output in JSON-formaat voor de producten die zijn gekocht en hun prijzen:\n\n```json\n{\n  "producten": [\n    {\n      "naam": "string 3 stuks",\n      "prijs": 8.59\n    },\n    {\n      "naam": "ondergoed 4 voor",\n      "prijs": 24.99\n    },\n    {\n      "naam": "string 4 voor",\n      "prijs": 24.99\n    }\n  ],\n  "totaal": 24.99\n}\n```', 'refusal': None}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 883, 'completion_tokens': 114, 'total_tokens': 997, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'system_fingerprint': 'fp_d02d531b47'}


In [ ]:
response_json = response.json()
print(response_json['choices'][0]['message']['content'])